In [48]:
import re
import json
import folium
import fileinput
import numpy as np
import pandas as pd
import seaborn as sns
import branca.colormap as cm
from datetime import datetime
from selenium import webdriver
from BindColorMap import BindColormap
from IPython.core.display import display
from folium.plugins import TimestampedGeoJson

In [84]:
class GeoMap():
    def __init__(self, coord_start = [46.519164, 6.566719]):
        self.geo_map = folium.Map(location = coord_start, control_scale = True, zoom_start = 11)
    
    def map_to_color(self, color_palette, list_of_values, min_range = None, max_range = None):
        if min_range == None:
            min_range = np.min(list_of_values)
        if max_range == None:
            max_range = np.max(list_of_values)
        
        intervals = np.linspace(min_range, max_range, len(color_palette)+1)

        mapped_colors = []
        for i in list_of_values:
            # we have N-1 colors
            for c in range (0, len(intervals) - 1):
                if c == 0 and i < intervals[c]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
                if c < len(intervals) and i >= intervals[c] and i < intervals[c + 1]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
                    break
                elif c == len(intervals) - 2 and i >= intervals[c]:
                    mapped_colors = np.append(mapped_colors, color_palette[c])
        return mapped_colors
    
    def generate_gradient_dict(self, color_palette):
        intervals = np.linspace(0, 1, len(color_palette)+1)
        
        # compute center of each interval:
        centers = np.round((intervals[1:] + intervals[:-1]) / 2, 3)
        
        gradient = {}
        for c in range(0, len(color_palette)):
            gradient.update({centers[c] : color_palette[c]})
        return gradient
    
    # displayColorData: a vector of values
    def geojson_layer(self, layer_name, jsonFile, displayColorData = None):
        layer_gjson = folium.FeatureGroup(name = layer_name)
        
        if displayColorData != None:
            color_palette = sns.color_palette('RdYlGn').as_hex()
            max_value = np.max(displayColorData)
            min_value = np.min(displayColorData)
            colors = self.map_to_color(color_palette, displayColorData.to_list(), min_range = None, max_range = None)
            colormap = cm.LinearColormap(['red','yellow','green'], vmin = min_value, vmax = max_value,
                                                      caption = layer_name).to_step(n = len(color_palette))
            
        print('\n')
        for i, geo_json in enumerate(jsonFile['features']):
            print('geojson_layer construction running... ' + str(int(np.ceil((i/len(jsonFile['features']))*100))) + '%', end='\r')
            
            if displayColorData != None:
                fill_col = colors[i]
                edges_color = 'white'
            else:
                fill_col = 'yellow'
                edges_color = 'blue'
            lj = folium.GeoJson(
                geo_json,
                name='geojson',
                style_function=lambda feature: {
                    'fillColor': fill_col,
                    'color' : edges_color,
                    'weight' : 1,
                    'fillOpacity' : 0.2,
                    }
                )
            popup = folium.Popup(geo_json['properties']['neighbourhood'])
            lj.add_child(popup)
            lj.add_to(layer_gjson)
            #airbnb_map.add_child(gj)
        
        if displayColorData != None:
            return [layer_gjson, colormap]
        else:
            return [layer_gjson, None]
        
        #layer_gjson.add_to(airbnb_map)
        #folium.LayerControl().add_to(airbnb_map)
        
    
    # displayType can be: Density, Points, Circles
    def pandas_layer(self, df, layer_name, displayType = 'Density', densityDataName = None):
        layer_pandas = folium.FeatureGroup(name = layer_name);
        
        if densityDataName != None:
            color_palette = sns.color_palette('RdYlGn').as_hex()
            colors = self.map_to_color(color_palette, df[densityDataName].tolist(), min_range = None, max_range = None)
            max_value = np.max(df[densityDataName])
            min_value = np.min(df[densityDataName])
            #colormap = cm.linear.RdYlGn.scale(min_value, max_value).caption(layer_name)
            colormap = cm.LinearColormap(['red','yellow','green'], vmin=min_value, vmax=max_value,
                                                      caption = layer_name).to_step(n = len(color_palette))
        
        print('\n')
        for index, row in df.iterrows():
            print('pandas_layer construction  running... ' + str(int(np.ceil((index/df.shape[0])*100))) + '%', end='\r')
            
            if displayType == 'Points' and densityDataName == None:
                lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                        radius = 2,
                                        color = 'red',
                                        fill_color = 'red')
                lp.add_to(layer_pandas)
            elif displayType == 'Points' and densityDataName != None:
                lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                        radius = 2,
                                        color = colors[index-1],
                                        fill_color = str(colors[index]))
                lp.add_to(layer_pandas)
            elif displayType == 'Circles' and densityDataName != None:
                lp = folium.CircleMarker([row['latitude'], row['longitude']],
                                        radius = 20 * row[densityDataName]/max_value,
                                        color = colors[index],
                                        fill_color = colors[index])
                lp.add_to(layer_pandas)
            elif displayType == 'Density' and densityDataName != None:
                points = df[['latitude', 'longitude', densityDataName]].values
                lp = folium.plugins.HeatMap(points, gradient = self.generate_gradient_dict(color_palette))
                lp.add_to(layer_pandas)
        
        if densityDataName != None:
            return [layer_pandas, colormap]
        else:
            return [layer_pandas, None]
    
    # layer_list is a list of [layer, color_map] or [layer, None] if the colormap is not present
    def add_layer_list(self, layer_list):
        for layer, cmap in layer_list:
            if cmap != None:
                self.geo_map.add_child(layer)
                self.geo_map.add_child(cmap)
                self.geo_map.add_child(BindColormap(layer, cmap))
            else:
                self.geo_map.add_child(layer)
        folium.LayerControl('topleft').add_to(self.geo_map)
            
    def add_tiles(self, tiles_list = ['OpenStreetMap', 'cartodbdark_matter'], tiles_names = ['Street Map', 'Dark Map']):
        # remove the default tile:
        del self.geo_map._children['openstreetmap']
        count = 0
        for tile, name_ in zip(tiles_list, tiles_names):
            # show first added tile when open the map
            if count == 0:
                lt = folium.TileLayer(tiles = tile, name = name_, show = True).add_to(self.geo_map)
            else:
                lt = folium.TileLayer(tiles = tile, name = name_).add_to(self.geo_map)
            count += 1
            
    def add_mini_map(self):
        minimap = folium.plugins.MiniMap()
        self.geo_map.add_child(minimap)
    
    def get_map(self):
        return self.geo_map
    
    def save_map(self, saving_destination_path, file_name):
        self.geo_map.save(saving_destination_path + '/' + file_name + '.html')
            

In [ ]:
# TODO: 
# check min and max in colormaps. Should for example fix them to the 0.05 and 0.95 quantiles
# check circles sizing system
# check density: should sample the data otherwise too slow in execution and too large output files -> too slow in display

In [73]:
input_csv_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/2019-09-14_Amsterdam_listings_detailed.csv'
geojson_file = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/DataSet/NaT_Amsterdam_neighbourhoods.geojson'
saving_path = '/Volumes/Disk2/Courses MA3/MA3 - ADA/AIRBNB data/Outputs'

df = pd.read_csv(input_csv_file, low_memory = False);
coord_start = [df.latitude.mean(), df.longitude.mean()]

with open(geojson_file) as f:
    jsonFile = json.load(f)

non_null = df[np.isfinite(df['review_scores_rating'])].reset_index()
non_null = non_null.loc[0:1000, :]

In [83]:
cart = GeoMap(coord_start);
cart.add_tiles()
layer1 = cart.pandas_layer(non_null, 'Global Rating Score', displayType = 'Points', densityDataName = 'review_scores_rating')
layer2 = cart.geojson_layer('Neighborhood', jsonFile, displayColorData = None)
cart.add_mini_map()
cart.add_layer_list([layer1, layer2])

output = cart.get_map()
cart.save_map(saving_path, 'test')
output




pandas_layer  running... 100%

